Author: Daniel Abadjiev  
&nbsp;&nbsp;&nbsp; with lots of help from Eliza  
Date: June 3, 2025  
Description: a notebook to test some stuff with smartpixels data and get a handle on the data format/machine learning general things.   

Will partially follow Eliza's github, will also follow this tutorial https://www.tensorflow.org/tutorials/keras/classification

In [1]:
from OptimizedDataGenerator4 import *
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

2025-06-06 11:13:35.191293: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-06 11:13:35.191816: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 11:13:35.194719: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 11:13:35.203347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749226415.217863  626071 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749226415.22

In [ ]:
data_dir = "../smartpixML/MuonColliderSim/Simulation_Output/";
tf_dir = "./tfRecDirDaniel"
labels_dir = "./labelsDirDaniel" #leftover 
dataFormatter = OptimizedDataGenerator(
    tf_records_dir=tf_dir,
    # data_directory_path=data_dir,
    # file_type="parquet",
    load_records=True,
    muon_collider=True,
    x_feature_description=['cluster','x_profile','x_size'],
    batch_size=3000,
    filteringBIB=True
    )
dataFormatter.tfrecord_filenames.size

365

In [4]:
print(dataFormatter)
# print(dataFormatter.x_features)
print(dataFormatter.x_feature_description)
# dataFormatter.x_features['x_profile']
print(dataFormatter.x_features['x_size'])
print(np.shape(dataFormatter.x_features['x_size']))
print(dataFormatter.labels)
# plt.plot(dataFormatter.labels)
model.fit(x=dataFormatter.x_features['x_size'],y=dataFormatter.labels)

['cluster', 'x_profile', 'x_size']


AttributeError: 'OptimizedDataGenerator' object has no attribute 'x_features'

In [2]:
validation_dir = "./tf_records1000Daniel/tfrecords_validation/"
train_dir = "./tf_records1000Daniel/tfrecords_train/"
x_feature_description: list = ['z_global']
trainODG = OptimizedDataGenerator(tf_records_dir=train_dir,load_records=True, x_feature_description=x_feature_description, muon_collider=True)
validationODG = OptimizedDataGenerator(tf_records_dir=validation_dir,load_records=True, x_feature_description=x_feature_description, muon_collider=True)
# trainODG._parse_tfrecord_fn()

In [10]:

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None,1), name="z_global"),
    # tf.keras.layers.Dense(128,activation='relu'),
    # tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
callbacks=[]
learningRates = [0.9,0.6,0.3,0.1,0.03,0.01,0.001,0.0001,0.00001,0.000001]
callbacks.append(tf.keras.callbacks.LearningRateScheduler(lambda epoch : learningRates[epoch]))
callbacks=[]
history = model.fit(x=trainODG,callbacks=callbacks,epochs=10)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 229ms/step - binary_accuracy: 0.4734 - loss: 0.7092
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 229ms/step - binary_accuracy: 0.5095 - loss: 0.6956
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 228ms/step - binary_accuracy: 0.5521 - loss: 0.6816
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 228ms/step - binary_accuracy: 0.5830 - loss: 0.6706
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 228ms/step - binary_accuracy: 0.5932 - loss: 0.6716
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 229ms/step - binary_accuracy: 0.6147 - loss: 0.6608
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 228ms/step - binary_accuracy: 0.6273 - loss: 0.6558
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 230ms/step - binary_accuracy: 0.6257 - loss: 0.6590
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 228ms/step - binary_accuracy: 0.6339 - loss: 0.6542
Epoch 10/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 229ms/step - binary_accuracy: 0.6226 - loss: 0.6599


In [9]:
test_loss, test_acc = model.evaluate(validationODG)
print("test accuracy: "+str(test_acc))

14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 232ms/step - binary_accuracy: 0.5740 - loss: 6.8665
test accuracy: 0.4445267915725708
